<a href="https://colab.research.google.com/github/merrymasti015/CAP2022GRP09/blob/main/PCAMZC321_Group09_FireEventDetection_VIDEO_VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Video File detection - addendum to PCAMZC321_Group09_FireEventDetection

In [ ]:
import keras
from keras.models import Sequential
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, InputLayer, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import pickle

from pathlib import Path
import cv2
import math
import os
from glob import glob
from scipy import stats as s


In [ ]:
from google.colab import drive
from IPython import display
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
RootFolder = "/content/gdrive/MyDrive/Fire_ImageDataSet/"
print(RootFolder)
output_directory = RootFolder+"/VDO_fire"  # where is my VDO file ?
ModelSaveFolder = RootFolder+"/ModelSave" # where is my model

/content/gdrive/MyDrive/Fire_ImageDataSet/


In [ ]:
video_title = "VDO-1"
input_video_file_path = f'{output_directory}/{video_title}.mp4'
input_video_file_path

'/content/gdrive/MyDrive/Fire_ImageDataSet//VDO_fire/VDO-1.mp4'

In [ ]:
## VGG-16 pre-trained model will be used for basemodel
# creating the base model of pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False)

58900480/58889256 [==============================] - 0s 0us/step


In [ ]:
#Video.from_file(input_video_file_path)
from ipywidgets import Video # Display VDO 
Video.from_file(input_video_file_path , width = 320 , height = 320)

Video(value=b'\x00\x00\x00 ftypisom\x00\x00\x02\x00isomiso2avc1mp41\x00\x00\x00\x08free\x00\xe3\xaa_mdat\x00\x…

In [ ]:
from ipywidgets import Widget
Widget.close_all()

In [ ]:
FramesVdo  = output_directory+"/VDOFrames"
if not os.path.exists(FramesVdo):
   os.makedirs(FramesVdo) 
FramesVdo

'/content/gdrive/MyDrive/Fire_ImageDataSet//VDO_fire/VDOFrames'

In [ ]:
# Reading the Video File using the VideoCapture Object
video_reader = cv2.VideoCapture(input_video_file_path)
# Set frames-per-second for capture
fps = round(video_reader.get(cv2.CAP_PROP_FPS))
## Total frame count in the VDO 
frame_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT)) -1 


duration = round(frame_count/fps)

print('fps = ' + str(fps))
print('number of frames = ' + str(frame_count))
print('duration (S) = ' + str(duration))

minutes = int(duration/60)
seconds = duration%60
print('duration (M:S) = ' + str(minutes) + ':' + str(seconds))  ## In Minute and seconds

video_reader.release()

fps = 25
number of frames = 1876
duration (S) = 75
duration (M:S) = 1:15


In [ ]:
    %%time
    flag =''
    image_height  = 224
    image_width   = 224
    
    EXTENSION = ".jpg"
    curr_frame = 0
    imageKount = 0
    prediction_images = []
    
    ## Check if VDOFrames is empty or Not --- If empty then frames will be stored else NOT
    ## OR we can delete all files in VDOFrames folder unconditionally 
    
    if not os.listdir(FramesVdo):
        print ("empty VDOFrames folder")
        flag = "empty"
    else:
        print ("Not empty")
    

    # Reading the Video File using the VideoCapture Object
    video_reader = cv2.VideoCapture(input_video_file_path)

    # Getting the width and height of the video 
    original_video_width = int(video_reader.get(cv2.CAP_PROP_FRAME_WIDTH))
    original_video_height = int(video_reader.get(cv2.CAP_PROP_FRAME_HEIGHT))



    prediction_images = []
    while True: 

        # Reading The Frame
        status, frame = video_reader.read() 

        if not status:
            break

        
        # Resize the Frame to fixed Dimensions
        resized_frame = cv2.resize(frame, (image_height, image_width))
        
        if flag == 'empty':
           
           temp = Path(input_video_file_path).stem # Find file name without extension
           filename ="/"+ temp+"_frame%d.jpg" % curr_frame
           Outimage = os.path.join(FramesVdo+filename)
           cv2.imwrite(Outimage, resized_frame)   # save frame as JPEG file
 
        
        
        
        
        
        resized_frame = image.img_to_array(resized_frame)
        # Normalize the resized frame by dividing it with 255 so that each pixel value then lies between 0 and 1
        normalized_frame = resized_frame / 255

        
        
        # converting the list to numpy array
        # appending the image to the image list
        prediction_images.append(normalized_frame)
        imageKount +=1
        curr_frame +=1
    ### End of while loop
    
    
    # converting the list to numpy array
    prediction_images = np.array(prediction_images)
    
    # shape of the array
    prediction_images.shape
    # extracting features for validation frames
    prediction_images = base_model.predict(prediction_images)
    print('Shape of prediction_images = ' ,prediction_images.shape)
    print('Total frame extracted      = ' , curr_frame)
    print('Total frame in the vdo     = ' ,imageKount)   


empty VDOFrames folder
Shape of prediction_images =  (1872, 7, 7, 512)
Total frame extracted      =  1872
Total frame in the vdo     =  1872
CPU times: user 30.3 s, sys: 2.57 s, total: 32.8 s
Wall time: 38.1 s


In [ ]:
# converting features in one dimensional array
prediction_images = prediction_images.reshape(prediction_images.shape[0], 7*7*512)

In [ ]:
# Normalize the array 
max = prediction_images.max()
prediction_images = prediction_images/max
prediction_images.shape

(1872, 25088)

In [ ]:
#Load Model
from tensorflow import keras

model_name = 'Step-1_VGG_model.h5'

model = keras.models.load_model(ModelSaveFolder+'/'+model_name)

In [ ]:
predict = []        
# predict_classes() function model in order to predict the class values for each instance in the array.
prediction = np.argmax(model.predict(prediction_images), axis=-1)
# prediction is an arrary containing frame by frame class prediction - here either Basketball', 'SoccerPenalty'

print(prediction)

[1 1 1 ... 0 0 0]


#### Pickle is the standard way of serializing objects in Python.

#### Use the pickle operation to serialize  machine learning algorithms and save the serialized format to a file.

#### Later we can load this file to deserialize your model and use it to make new predictions.

#### Serialization is the process of converting an object into a stream of bytes to store the object or transmit it to memory, or a file.

In [ ]:
import pickle

with open('Step-2_FireDetection_VGG16_Feature', 'wb') as fp:
    pickle.dump(prediction, fp)

In [ ]:
#To read it back:
with open ('Step-2_FireDetection_VGG16_Feature', 'rb') as fp:
    prediction = pickle.load(fp)

In [ ]:
prediction

array([1, 1, 1, ..., 0, 0, 0])

### Function To Predict on Live Videos 

In [ ]:
input_video_file_path

'/content/gdrive/MyDrive/Fire_ImageDataSet//VDO_fire/VDO-1.mp4'

In [ ]:
# Constructing The OutputVideo Path
output_video_file_path = f'{output_directory}/{video_title}_VGG16_FeatureOutput.mp4'

print(output_video_file_path)

/content/gdrive/MyDrive/Fire_ImageDataSet//VDO_fire/VDO-1_VGG16_FeatureOutput.mp4


In [ ]:
trainFrames  = pd.read_csv(RootFolder+"/OutputFiles/Train_FramePathLocation.csv")

trainFrames.head()
# Get the unique values of 'class' column - How many distinct classes
model_class = trainFrames['class'].unique().tolist()
print(model_class)
model_output_size = len(model_class)
print(model_output_size)


['Neutral', 'Fire']
2


In [ ]:
prediction[2]

1

In [ ]:
x = len(prediction)
print(x)
# Accessing The Class Name using predicted label.
predicted_class_name = model_class[prediction[2]]
predicted_class_name

1872


'Fire'

In [ ]:
print('Input path:'+input_video_file_path)
print('Output path:'+output_video_file_path)

Input path:/content/gdrive/MyDrive/Fire_ImageDataSet//VDO_fire/VDO-1.mp4
Output path:/content/gdrive/MyDrive/Fire_ImageDataSet//VDO_fire/VDO-1_VGG16_FeatureOutput.mp4


In [ ]:
def predict_on_live_video(video_file_path, output_file_path,Algo):

    # Initialize a Deque Object with a fixed size which will be used to implement moving/rolling average functionality.
    #predicted_labels_probabilities_deque = deque(maxlen = window_size)
    
    frame_no = -1

    # Reading the Video File using the VideoCapture Object
    video_reader = cv2.VideoCapture(video_file_path)

    # Getting the width and height of the video 
    original_video_width = int(video_reader.get(cv2.CAP_PROP_FRAME_WIDTH))
    original_video_height = int(video_reader.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Writing the Overlayed Video Files Using the VideoWriter Object
    video_writer = cv2.VideoWriter(output_file_path, cv2.VideoWriter_fourcc('M', 'P', '4', 'V'), 24, (original_video_width, original_video_height))

    while True: 

        # Reading The Frame
        status, frame = video_reader.read() 

        if not status:
            break

        frame_no = frame_no+1
        
        # some frames with black - skip those
        ## images in openCV (or in your case frames) are represented as a numpy array, 
        ## they can be averaged for low values (which represent black frames).
        
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  ## Convert to grey image (black and white)
        if np.average(gray) < 20:  ## if it dark screen , skip 
        # skips an iteration, so the frame isn't saved
          continue
        

           
        # Accessing The Class Name using prediction list.
        predicted_class_name = model_class[prediction[frame_no]]
        #print(predicted_class_name)
        
        
        # Overlaying Class Name Text Ontop of the Frame
        
        cv2.putText(frame, predicted_class_name+'_(' + Algo + ')', (00, 200), cv2.FONT_HERSHEY_DUPLEX, 3.0, (0, 0, 255), 3)

        #cv2.putText(frame, predicted_class_name, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        #cv2.putText(frame, avg_prob, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
        
        #cv2.putText(frame, 
                    #avg_prob, 
                    #(10, 100),   # bottomLeftCornerOfText
                    #cv2.FONT_HERSHEY_SIMPLEX, 
                    #1, 
                    #(0, 0, 255), 
                    #2)
                       
        # Writing The Frame
        video_writer.write(frame)
    
    # Closing the VideoCapture and VideoWriter objects and releasing all resources held by them. 
    video_reader.release()
    video_writer.release()

In [ ]:
Model_apply = 'VGG16_Feature'
# Calling the predict_on_live_video method to start the Prediction.
predict_on_live_video(input_video_file_path, output_video_file_path,Model_apply)